In [97]:
# Import necessary libraries
import ephem
import math
from datetime import timedelta
from datetime import datetime


# get_passes() function definition
def get_passes(observer, tle, start_time, num_passes=None, duration=None):
    """Config obs and sat, Return pass data for all passes in given interval.

    Arguments:
    observer -- 4 element list containing desired [name,lat,lon,alt]
    tle -- 3 element list containing desired tle [line0,line1,line2]
    start_time -- ephem.date string formatted 'yyyy/mm/dd hr:min:sec'
    num_passes -- integer number of desired passes (defualt None)
    duration -- float number of hours or fraction of hours (default None)

    Specify either num_passes or duration.
    If both, use num_passes.
    If neither, find passes for next 24 hours.
    """

    obs_name, obs_lat, obs_lon, obs_alt = observer
    tle_line0, tle_line1, tle_line2 = tle

    # Set up location of observer
    ground_station = ephem.Observer()
    ground_station.name = obs_name                # name string
    ground_station.lon = obs_lon                  # in degrees (+E)
    ground_station.lat = obs_lat                  # in degrees (+N)
    ground_station.elevation = obs_alt            # in meters
    ground_station.date = ephem.date(start_time)  # in UTC

    # Read in most recent satellite TLE data
    sat = ephem.readtle(tle_line0, tle_line1, tle_line2)

    contacts = []

    if num_passes is not None and duration is None:
        # if only num_passes specified
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(days = set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
                contacts.append(pass_data)
        except ValueError:
            # No (more) visible passes
            pass
        return contacts

    if num_passes is None and duration is not None:
        # if only duration specified
        try:
            end_time = ephem.date(ground_station.date+duration*ephem.hour)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data

    if num_passes is not None and duration is not None:
        # if both are specified, use num_passes
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data

    if num_passes is None and duration is None:
        # if neither are specified, get passes for the next 24 hours
        try:
            end_time = ephem.date(ground_station.date+1)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = timedelta(set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data = {
                    'start' : rise_time.datetime().ctime(),
                    'end' : set_time.datetime().ctime(),
                    'duration' : str(pass_duration),
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data


In [98]:
import requests

In [99]:
# itertools has lots of things that deal with looping
# "generators" and "iterators" are really nice concepts that
#can be used in place of most usage of using indexes in a loop
#because usually we aren't actually interested in counting the lines
from itertools import islice
data = []
with open('amateur.txt') as f:
    #we don't know we've reached the end of the file
    #until we actually reached it
    while True:
        #an iterator that returns the next N lines and stops
        tripleline = islice(f, 3)
        #loop over these N lines, removing trailing spaces and \n
        # this is called a "list comprehension"
        # it is pretty much exactly the same as
        #    tle = []
        #    for line in tripleline:
        #      tle.append(x.rstrip())
        #
        tle = [x.rstrip() for x in tripleline]
        
        #only accept complete data
        #the end of the file *should* have len(tle)==0 but
        #this also handles extra junk at the end
        if len(tle) == 3:
            data.append(tle)
        else:
            break

# now, data is a list of 3-element lists of strings

In [100]:
# get NORAD ID for each sat
print(data[0][2][2:7])
print(data[1][2][2:7])

07530
14781


In [103]:
vu = ['Valparaiso University', '41.4639', '-87.0439', 245.089]
start_time = '2017/6/8 12:00:00'
num_passes = 2
duration = None
vu_passes = {}
i = 0

for tle in data:
    # use NORAD ID as key for each satellite
    # value is list of passes, where each pass is a dictionary of data
    vu_passes[data[i][2][2:7]] = get_passes(vu, tle, start_time, num_passes=num_passes, duration=duration)
    i = i + 1

print(vu_passes['25544'])

[{'rise_az': 190.456669458409, 'end': 'Thu Jun  8 19:35:05 2017', 'duration': '0:09:29.955764', 'set_az': 70.48850846101539, 'start': 'Thu Jun  8 19:25:35 2017'}, {'rise_az': 238.95741928294103, 'end': 'Thu Jun  8 21:11:59 2017', 'duration': '0:11:03.015994', 'set_az': 53.80764288509183, 'start': 'Thu Jun  8 21:00:56 2017'}]


In [104]:
import pprint
pprint.pprint(vu_passes)

{'07530': [{'duration': '0:12:41.902490',
            'end': 'Thu Jun  8 13:18:54 2017',
            'rise_az': 3.1217117066147058,
            'set_az': 291.59949038617015,
            'start': 'Thu Jun  8 13:06:12 2017'},
           {'duration': '0:09:47.030825',
            'end': 'Thu Jun  8 16:52:59 2017',
            'rise_az': 53.02774114980851,
            'set_az': 358.85267484113393,
            'start': 'Thu Jun  8 16:43:12 2017'}],
 '14781': [{'duration': '0:10:19.870162',
            'end': 'Thu Jun  8 13:26:56 2017',
            'rise_az': 36.77677451043472,
            'set_az': 140.19885346989122,
            'start': 'Thu Jun  8 13:16:36 2017'},
           {'duration': '0:13:17.266591',
            'end': 'Thu Jun  8 15:05:01 2017',
            'rise_az': 9.724222283617193,
            'set_az': 199.13856389354723,
            'start': 'Thu Jun  8 14:51:43 2017'}],
 '20442': [{'duration': '0:11:54.066656',
            'end': 'Thu Jun  8 16:59:57 2017',
            'ris

186.9450779608087,
            'start': 'Thu Jun  8 13:34:21 2017'}],
 '40901': [{'duration': '0:10:52.042873',
            'end': 'Thu Jun  8 13:14:48 2017',
            'rise_az': 356.6121542273163,
            'set_az': 227.54405023644466,
            'start': 'Thu Jun  8 13:03:56 2017'},
           {'duration': '0:10:25.490901',
            'end': 'Thu Jun  8 22:21:12 2017',
            'rise_az': 125.96939742859861,
            'set_az': 6.318460298337994,
            'start': 'Thu Jun  8 22:10:46 2017'}],
 '40902': [{'duration': '0:10:58.730243',
            'end': 'Thu Jun  8 13:07:00 2017',
            'rise_az': 357.341591109917,
            'set_az': 225.86226641749076,
            'start': 'Thu Jun  8 12:56:01 2017'},
           {'duration': '0:10:17.572220',
            'end': 'Thu Jun  8 22:13:20 2017',
            'rise_az': 124.25765639994503,
            'set_az': 7.123622634492961,
            'start': 'Thu Jun  8 22:03:02 2017'}],
 '40903': [{'duration': '0:11:14.6961